In [1]:
%load_ext autoreload
%autoreload 2
import sys; sys.path.insert(0, '..') # add parent folder path where lib folder is

import optuna
import joblib
import dask.distributed
import dask_optuna



In [2]:
from utils import dasker, helper, config
import pandas as pd

/home/conda/admin/1c9d9fbeff3e46180d6a8a009af482ef5e93abe04420c8eddea14e360f402824-20221103-192738-068973-22-wasif-dev/lib/python3.9/site-packages/cupy/_environment.py:437: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy, cupy-cuda115

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''


In [3]:
client = dasker.get_global_client()

connected to cluster dev.cc639015ed5141b48084bb654062e917


/home/conda/admin/1c9d9fbeff3e46180d6a8a009af482ef5e93abe04420c8eddea14e360f402824-20221103-192738-068973-22-wasif-dev/lib/python3.9/site-packages/distributed/client.py:1348: VersionMismatchWarning: Mismatched versions found

+-------------+-----------+-----------+-----------+
| Package     | client    | scheduler | workers   |
+-------------+-----------+-----------+-----------+
| dask        | 2022.10.2 | 2022.04.2 | 2022.04.2 |
| distributed | 2022.10.2 | 2022.4.2  | 2022.4.2  |
+-------------+-----------+-----------+-----------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [4]:
X, y = helper.get_dd_covid_dataset()

In [13]:
import xgboost as xgb



def objective(trial, X, y ):
    
#    X = X.drop(['location'], axis = 1)
    
    # X, y = helper.get_dd_covid_dataset()
    param = {
        "objective": 'count:poisson',
        "booster": trial.suggest_categorical("booster", ["gbtree", "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-3, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-3, 1.0, log=True),
    }

    
    
    dtrain = xgb.DMatrix(X, label=y)
    

    model = xgb.train(param, dtrain, num_boost_round = 10, verbose_eval = 1)
    
    
    return 15.0

In [15]:
# import uniform
import random


X, y = helper.get_covid_dataset()
# x = random.random() #uniform(2.5, 10.0)
# y = random.random() #uniform(2.5, 10.0)


func = lambda trial: objective(trial, X, y)
# Create a study using Dask-compatible storage
# dtrain = xgb.dask.DaskDMatrix(client, X, y)
storage = dask_optuna.DaskStorage()
study = optuna.create_study(storage=storage)
# Optimize in parallel on your Dask cluster
with joblib.parallel_backend("dask",  wait_for_workers_timeout=1200):
    study.optimize(func, n_trials=10, n_jobs=2)
    print(f"best_params = {study.best_params}")

2022-11-08 03:50:29,484 - distributed.protocol.pickle - INFO - Failed to deserialize b'\x80\x04\x95\x16\x06\x00\x00\x00\x00\x00\x00\x8c\x15distributed.scheduler\x94\x8c\x0cKilledWorker\x94\x93\x94\x8cFbatch_of__optimize_sequential_1_calls-078e5e1f8dfb439fb55233ef3b152fb2\x94h\x00\x8c\x0bWorkerState\x94\x93\x94)\x81\x94N}\x94(\x8c\x07address\x94\x8c\x17tls://10.10.6.148:39127\x94\x8c\x03pid\x94K\x1a\x8c\x04name\x94\x8c2dask-worker-cc639015ed5141b48084bb654062e917-nt2gr\x94\x8c\x08nthreads\x94K\x03\x8c\x0cmemory_limit\x94\x8a\x05\x00\x00\x00\xe0\x03\x8c\x0flocal_directory\x94\x8c-/home/wasif/dask-worker-space/worker-6p2sjipo\x94\x8c\x08services\x94}\x94\x8c\tdashboard\x94MS"s\x8c\x08versions\x94}\x94\x8c\x05nanny\x94\x8c\x17tls://10.10.6.148:39893\x94\x8c\x06status\x94\x8c\x10distributed.core\x94\x8c\x06Status\x94\x93\x94\x8c\x06closed\x94\x85\x94R\x94\x8c\x05_hash\x94\x8a\x08\xbe\xd3\x1e\xe3~U\x07\xb3\x8c\x06nbytes\x94K\x00\x8c\toccupancy\x94K\x00\x8c\x15_memory_unmanaged_old\x94K\x00\x

Exception: b'\x80\x04\x95r\x05\x00\x00\x00\x00\x00\x00\x8c\x15distributed.scheduler\x94\x8c\x0cKilledWorker\x94\x93\x94\x8cFbatch_of__optimize_sequential_1_calls-8c54b5b9166941d89c698b737c806c6d\x94h\x00\x8c\x0bWorkerState\x94\x93\x94)\x81\x94N}\x94(\x8c\x07address\x94\x8c\x17tls://10.10.41.43:33891\x94\x8c\x03pid\x94K\x1a\x8c\x04name\x94\x8c2dask-worker-cc639015ed5141b48084bb654062e917-946x9\x94\x8c\x08nthreads\x94K\x03\x8c\x0cmemory_limit\x94\x8a\x05\x00\x00\x00\xe0\x03\x8c\x0flocal_directory\x94\x8c-/home/wasif/dask-worker-space/worker-c3045eug\x94\x8c\x08services\x94}\x94\x8c\tdashboard\x94MS"s\x8c\x08versions\x94}\x94\x8c\x05nanny\x94\x8c\x17tls://10.10.41.43:33139\x94\x8c\x06status\x94\x8c\x10distributed.core\x94\x8c\x06Status\x94\x93\x94\x8c\x06closed\x94\x85\x94R\x94\x8c\x05_hash\x94\x8a\x08y\x12U\xe2\xed\x9e\xee|\x8c\x06nbytes\x94K\x00\x8c\toccupancy\x94K\x00\x8c\x15_memory_unmanaged_old\x94K\x00\x8c\x19_memory_unmanaged_history\x94\x8c\x0bcollections\x94\x8c\x05deque\x94\x93\

/home/conda/admin/1c9d9fbeff3e46180d6a8a009af482ef5e93abe04420c8eddea14e360f402824-20221103-192738-068973-22-wasif-dev/lib/python3.9/site-packages/dask_gateway/client.py:1014: RuntimeWarning: coroutine 'rpc.close_rpc' was never awaited
  self.scheduler_comm.close_rpc()
